### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Content-Based Filtering</a></li>
    </ol>
</div>
<br>


<a id="ref2"></a>
# Preprocessing


In [1]:
import pandas as pd 
from math import sqrt
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
movie_df = pd.read_csv(r"T:\My Drive\000 ihsan\02 Courses\003GIT\MYProject\Ml_project_IBM\Content Based Filtering\moviedataset\ml-latest\movies.csv")
rating_df = pd.read_csv(r"T:\My Drive\000 ihsan\02 Courses\003GIT\MYProject\Ml_project_IBM\Content Based Filtering\moviedataset\ml-latest\ratings.csv")

In [3]:
movie_df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating_df.head(5)

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [23]:
movie_df["Year"] = movie_df.title.str.extract('(\(\d\d\d\d\))',expand=False) # \( open ,\) close,  Extracts a four-digit year - Toy Story (1995) - (1995),str.extract used to extract specific substrings from a Series
movie_df["Year"] = movie_df.Year.str.extract('(\d\d\d\d)',expand=False) #Removes the parentheses() from the year column,
movie_df['title'] = movie_df.title.str.replace('(\(\d\d\d\d\))','',regex=True) # regex=True Regular String,  (\n → newline)
movie_df['title'] = movie_df["title"].apply(lambda x: x.strip()) # " Toy Story " - "Toy Story", "  hello  ".strip()  # Output: "hello"

movie_df.head(2)

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\admin\AppData\Local\Temp\ipykernel_16912\50843332.py:1: SyntaxWarning: invalid escape sequence '\('
  movie_df["Year"] = movie_df.title.str.extract('(\(\d\d\d\d\))',expand=False) # \( open ,\) close,  Extracts a four-digit year - Toy Story (1995) - (1995),str.extract used to extract specific substrings from a Series
C:\Users\admin\AppData\Local\Temp\ipykernel_16912\50843332.py:2: SyntaxWarning: invalid escape sequence '\d'
  movie_df["Year"] = movie_df.Year.str.extract('(\d\d\d\d)',expand=False) #Removes the parentheses() from the year column,
C:\Users\admin\AppData\Local\Temp\ipykernel_16912\50843332.py:3: SyntaxWarning: invalid escape sequence '\('
  movie_df['title'] = movie_df.t

,movieId,title,genres,Year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",NaN


With that, let's also split the values in the __Genres__ column into a __list of Genres__ to simplify for future use. This can be achieved by applying Python's split string function on the correct column.


In [6]:
movie_df["genres"] = movie_df.genres.str.split('|')
movie_df.head(3)

,movieId,title,genres,Year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995


In [7]:
# creates an independent copy of movies_df.
movie_genres_df = movie_df.copy() # moviesWithGenres_df = movies_df - assighnment , creates an independent copy of movies_df.

for index,row in movie_df.iterrows(): # Iterates through each row of the movie_df DataFrame
    for genre in row['genres']: # each row (representing a movie), this line iterates through the list of genres (row['genres']), assuming each movie can have multiple genres.
        movie_genres_df.at[index,genre] = 1 # updates the movie_genres_df DataFrame by setting the value at the position specified by index (the movie) and genre (the column) to 1.

movie_genres_df = movie_genres_df.fillna(0) # Ensures that all cells in the DataFrame have a value
movie_genres_df.head(5)


,movieId,title,genres,Year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
rating_df.head(3)

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435


In [9]:
#Drop removes a specified row or column from a dataframe
rating_df = rating_df.drop('timestamp',axis=1)
rating_df.head(5)

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>
# Content-Based recommendation system


Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the __userInput__. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [10]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 

inputmovies = pd.DataFrame(userInput)
inputmovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [11]:
inputid = movie_df[movie_df['title'].isin(inputmovies['title'].tolist())] # filters the movies_df DataFrame to only include the rows (movies) whose titles are present in the inputMovies['title'] list
inputmovies = pd.merge(inputid,inputmovies) # Merges the inputId DataFrame (filtered movies from movies_df) with the inputMovies DataFrame (user input).
inputmovies = inputmovies.drop('genres',axis=1).drop('Year',axis=1) # Drops the genres and year columns from the inputMovies DataFrame, as they are not needed

inputmovies.head(5)

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [24]:
usermovie = movie_genres_df[movie_genres_df['movieId'].isin(inputmovies['movieId'].tolist())]
usermovie.head(3)

,movieId,title,genres,Year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [25]:
usermovie = usermovie.reset_index(drop=True) # Resetting the index to avoid future issues
#Dropping unnecessary issues due to save memory and to avoid issues
usergenertable = usermovie.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('Year',axis=1)
usergenertable.head(3)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling the Pandas "dot" function.


In [14]:
inputmovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [26]:
#Dot produt to get weight
userprofile = usergenertable.transpose().dot(inputmovies['rating']) #  transpose() - rows become columns and vice versa,.dit - a= a,b,c b = 1,2,3 - a.b - a1,b2,c2
userprofile.head(3)


Adventure    10.0
Animation     8.0
Children      5.5
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [16]:
genretable = movie_genres_df.set_index(movie_genres_df['movieId']) # movieId column as the index of the DataFrame
genretable = genretable.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('Year',axis=1)
genretable.head(3)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
genretable.shape

(34208, 20)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [27]:
#Multiply the genres by the weights and then take the weighted average,Normalizes the weighted scores so they become averages rather than raw totals

recommendation_table_df = ((genretable * userprofile).sum(axis=1)) / (userprofile.sum()) # sum(axis=1)-summation should occur across columns (horizontally for each row).

recommendation_table_df.head(3)

movieId
1    0.594406
2    0.293706
3    0.188811
dtype: float64

In [19]:
recommendation_table_df = recommendation_table_df.sort_values(ascending=False) # ts a Series or DataFrame by its values , sorting should be in descending order
recommendation_table_df.head(5)

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

### Now here's the recommendation table!


In [22]:
# .keys() retrieves the index values of these rows, which are the movieIds

movie_df[movie_df['movieId'].isin(recommendation_table_df.head(20).keys())]


,movieId,title,genres,Year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2902,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11716,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003
